In [48]:
import math
import random

In [49]:
# the core class for creating a computational graph and performing automatic differentiation
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    """it returns a new Value object that is the tanh of the original Value object, and it also sets up the backward function for the tanh operation in it's object"""
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1) # tanh function is (e^(2x) - 1)/(e^(2x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad # the derivative of tanh is 1 - tanh^2, and we multiply by out.grad to chain the gradients together i.e dy/dx = dy/dt * dt/dx, where dy/dt is out.grad and dt/dx is 1 - t^2
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [50]:
# defining the building blocks for a neural network: Neuron, Layer, and MLP (Multi-Layer Perceptron)
class Neuron:
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]


In [51]:
# example usage, initial simple forward pass
x = [2.0, 3.0, -1.0]
nn = MLP(3, [4, 4, 1]) # means 3 inputs, 2 hidden layers with 4 neurons each, and 1 output
nn(x).data

0.917422717294259

In [52]:
# training loop for the neural network on a simple dataset
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
nn = MLP(3, [4, 4, 1])

for k in range(20):
  # forward pass
  ypred = [nn(x) for x in xs]
  loss = sum((yout - y)**2 for y, yout in zip(ys, ypred))
  
  # backward pass
  for p in nn.parameters():
    p.grad = 0.0
  loss.backward()
  
  # updating parameters
  for p in nn.parameters():
    p.data += -0.1 * p.grad # minimizing the loss function
  
  print(f"iteration: {k}, loss: {loss.data}")



iteration: 0, loss: 9.239348818926672
iteration: 1, loss: 4.908350331140376
iteration: 2, loss: 4.628393392019531
iteration: 3, loss: 4.736545931565928
iteration: 4, loss: 4.806967005826005
iteration: 5, loss: 4.677491566092475
iteration: 6, loss: 3.660132816005278
iteration: 7, loss: 2.9541631541059843
iteration: 8, loss: 1.2329408594008675
iteration: 9, loss: 0.20553435230442574
iteration: 10, loss: 0.09442264991834232
iteration: 11, loss: 0.07021856297587813
iteration: 12, loss: 0.056853500404995906
iteration: 13, loss: 0.047878123112210576
iteration: 14, loss: 0.041309356649589325
iteration: 15, loss: 0.036263523552621156
iteration: 16, loss: 0.0322612806648664
iteration: 17, loss: 0.02901105480329963
iteration: 18, loss: 0.026322201008907302
iteration: 19, loss: 0.024063753585980882


In [53]:
# inference with the trained neural network
ypred = [nn(x) for x in xs]
print(ys)
print([y.data for y in ypred])

[1.0, -1.0, -1.0, 1.0]
[0.9140421518949211, -0.9324978966608405, -0.9053253838356398, 0.9648737856134161]
